In [1]:
import pandas as pd
import ast

In [2]:
data = pd.read_csv('data/CombinedMultipleData.csv')
data

,QuestionText,QType,Answer,AnswerChoices,Year,Tags,Description
0,What will the end-of-day closing value for the...,mc,6.40 or more,"['Less than 6.30', 'Between 6.30 and 6.35, inc...",2016,"['Finance', 'Economic Indicators']",Outcome will be determined by the end-of-day c...
1,How many seats will the Justice and Developmen...,mc,A majority,"['A majority', 'A plurality', 'Not a plurality']",2015,"['Elections and Referenda', 'Non-US Politics']",The Justice and Development Party (AKP) failed...
2,Will Congress pass a resolution disapproving t...,mc,No,"['No', 'Yes, but the resolution will be vetoed...",2015,"['Foreign Policy', 'Security and Conflict', 'U...",In accordance with the Iran Nuclear Agreement ...
3,Who will win the presidential election in Arge...,mc,Mauricio Macri,"['Someone else', 'Sergio Massa', 'Mauricio Mac...",2015,"['Leader Entry/Exit', 'Non-US Politics']","Elections are scheduled for 25 October, with a..."
4,How many seats in Poland's Sejm will PiS (Law ...,mc,Majority,"['Majority', 'Plurality', 'Not a Plurality']",2015,['Non-US Politics'],A PiS victory in Poland's parliamentary electi...
...,...,...,...,...,...,...,...
921,Jun 2022,mc,More than or equal to 90 but less than 100,"['Less than 40', 'More than or equal to 40 but...",2022,[],The imposition of substantial economic sanctio...
922,What will be the price per ton of aluminum on ...,mc,Less than $3000,"['Less than $3000', 'More than or equal to $30...",2022,"['Microelectronics', 'EA College Tournament']",An aluminum shortage has seen the price of alu...
923,What will be the end of day closing price of t...,mc,Less than $240,"['Less than $240', 'More than or equal to $240...",2022,['EA College Tournament'],This question will resolve using the “close” p...
924,What will be the price of crude oil futures on...,mc,More than or equal to $105 but less than $120,"['Less than $75', 'More than or equal to $75 b...",2022,['EA College Tournament'],This question will be resolved using the “clos...


In [3]:
startFrom, endAt = 0, 926

In [4]:
questions = list(data['QuestionText'])
options = [ast.literal_eval(x) for x in list(data['AnswerChoices'])]

In [5]:
from gpt4all import GPT4All
from tqdm import tqdm

def saveAnswers(questions, start, end):
    predictions = pd.DataFrame()
    predictions['predictions'] = questions
    predictions.to_csv(f'data/WizardMulti{start}-{end}.csv')

model = GPT4All("wizardlm-13b-v1.2.Q4_0.gguf")
outputs = []
for i in tqdm(range(startFrom, endAt)):
    with model.chat_session():
        opts = '\n'.join(options[i])
        prompt = f"### Instruction: \nAnswer the question below only with the options provided. Only include one of the options in your response. Feel free to hallucinate\n ### Question: \n{questions[i]} \n ### Options: \n{opts}\n ### Response:\n"
        re = model.generate(prompt, temp=0.7)
        outputs.append(re)
        if i % 100 == 0:
            saveAnswers(outputs, startFrom, i)

100%|██████████████████████████████████████████████████████████████████████████████| 926/926 [9:45:23<00:00, 37.93s/it]


In [6]:
predictions = pd.DataFrame()
predictions['predictions'] = outputs
predictions.to_csv(f'data/WizardMulti0-925.csv')